In [2]:
import pandas as pd
import numpy as np
import os
import glob

In [113]:
#read all files in folder 'Data' in,
#extract 'Nameplate Capacity (MW)' of operating and standby capacity for coal and gas
#and save sums in DataFrame

def extract_nameplate_capacity():
    path = "Data"
    files = glob.glob(os.path.join(path, "*.xlsx"))
    #DataFrame which will contain sums of generating capacities
    df_gen = pd.DataFrame(columns=['Year','Month','Capacity Gas (MW)','Capacity Coal (MW)','Capacity Oil (MW)'])
    counter = 0

    for f in files:
        counter += 1

        #read files (f = 'Data\\january_generator2022.xlsx')
        #get month and year
        month = f.split("\\")[-1].split("_")[0]
        month = month.capitalize()
        year = f.split(".")[0][-4:]
        if int(year) > 2020:
            rows_to_skip = 2
        else:
            rows_to_skip = 1
        #montly generator data
        gen = pd.read_excel(f, skiprows=rows_to_skip)

        print(f)

        gen.columns = [x.replace("\n", "") for x in gen.columns.to_list()]
        gen.columns = [x.replace(" ", "") for x in gen.columns.to_list()]

        #ISO = New York
        #only operating capacity or in standby
        NY_gen = gen[gen['BalancingAuthorityCode']=='NYIS']
        NY_gen = NY_gen[(NY_gen['Status']=='(OP) Operating') | (NY_gen['Status']=='(SB) Standby/Backup: available for service but not normally used')]
        #sum capacity up by resource type (only gas and coal)
        NY_gen_grouped = NY_gen.groupby('EnergySourceCode')
        #gas: 'BFG','NG','OG'
        gas_flag = False
        for (name, group) in NY_gen_grouped:
            if name in ['BFG','NG','OG']:
                gas_flag = True
        if gas_flag == True:
            NY_gen_gas = pd.concat([group for (name, group) in NY_gen_grouped if name in ['BFG','NG','OG']])
            NY_gen_gas['NameplateCapacity(MW)'] = NY_gen_gas['NameplateCapacity(MW)'].replace(" ",0)
            NY_gen_gas['NameplateCapacity(MW)'] = NY_gen_gas['NameplateCapacity(MW)'].astype(float)
            NY_gen_gas_sum = NY_gen_gas['NameplateCapacity(MW)'].sum()
        else:
            NY_gen_gas_sum = 0
        #coal: 'ANT','BIT','LIG','SGC','SUB','WC','RC'
        coal_flag = False
        for (name, group) in NY_gen_grouped:
            if name in ['ANT','BIT','LIG','SGC','SUB','WC','RC']:
                coal_flag = True
        if coal_flag == True:
            NY_gen_coal = pd.concat([group for (name, group) in NY_gen_grouped if name in ['ANT','BIT','LIG','SGC','SUB','WC','RC']])
            NY_gen_coal['NameplateCapacity(MW)'] = NY_gen_coal['NameplateCapacity(MW)'].replace(" ",0)
            NY_gen_coal['NameplateCapacity(MW)'] = NY_gen_coal['NameplateCapacity(MW)'].astype(float)
            NY_gen_coal_sum = NY_gen_coal['NameplateCapacity(MW)'].sum()
        else:
            NY_gen_coal_sum = 0
        #petroleum products: 'DFO','JF','KER','PC','PG','RFO','SGP','WO'
        oil_flag = False
        for (name, group) in NY_gen_grouped:
            if name in ['DFO','JF','KER','PC','PG','RFO','SGP','WO']:
                oil_flag = True
        if oil_flag == True:
            NY_gen_oil = pd.concat([group for (name, group) in NY_gen_grouped if name in ['DFO','JF','KER','PC','PG','RFO','SGP','WO']])
            NY_gen_oil['NameplateCapacity(MW)'] = NY_gen_oil['NameplateCapacity(MW)'].replace(" ",0)
            NY_gen_oil['NameplateCapacity(MW)'] = NY_gen_oil['NameplateCapacity(MW)'].astype(float)
            NY_gen_oil_sum = NY_gen_oil['NameplateCapacity(MW)'].sum()
        else:
            NY_gen_oil_sum = 0

        #save generating capacity data in DataFrame
        new_row = pd.DataFrame(data={'Year':year, 'Month':month, 'Capacity Gas (MW)':NY_gen_gas_sum, 
                                     'Capacity Coal (MW)':NY_gen_coal_sum, 'Capacity Oil (MW)':NY_gen_oil_sum}, index=[counter])
        df_gen = pd.concat([df_gen.reset_index(drop=True), new_row.reset_index(drop=True)])
        #save every 10th run as .csv file (still unsorted version, perliminary)
        if counter%10 == 0:
            df_gen.to_csv('Montly_Generating_Capacity_EIA.csv', index=False)  


    #sort by year and month
    months_in_order = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
    df_gen['Month'] = pd.Categorical(df_gen['Month'], categories=months_in_order, ordered=True)
    df_gen.sort_values(by=['Year','Month'], inplace=True)

    #save as .csv file (sorted, complete data)
    df_gen.to_csv('Montly_Generating_Capacity_EIA.csv', index=False) 

    return df_gen

Data\april_generator2017.xlsx
Data\april_generator2018.xlsx
Data\april_generator2019.xlsx
Data\april_generator2020.xlsx
Data\april_generator2021.xlsx
Data\april_generator2022.xlsx
Data\august_generator2017.xlsx
Data\august_generator2018.xlsx
Data\august_generator2019.xlsx
Data\august_generator2020.xlsx
Data\august_generator2021.xlsx
Data\august_generator2022.xlsx
Data\december_generator2017.xlsx
Data\december_generator2018.xlsx
Data\december_generator2019.xlsx
Data\december_generator2020.xlsx
Data\december_generator2021.xlsx
Data\december_generator2022.xlsx
Data\february_generator2017.xlsx
Data\february_generator2018.xlsx
Data\february_generator2019.xlsx
Data\february_generator2020.xlsx
Data\february_generator2021.xlsx
Data\february_generator2022.xlsx
Data\february_generator2023.xlsx
Data\january_generator2017.xlsx
Data\january_generator2018.xlsx
Data\january_generator2019.xlsx
Data\january_generator2020.xlsx
Data\january_generator2021.xlsx
Data\january_generator2022.xlsx
Data\january_

,Year,Month,Capacity Gas (MW),Capacity Coal (MW),Capacity Oil (MW)
25,2017,January,24444.9,1206.5,3832.8
18,2017,February,24265.9,1318.5,3700.0
44,2017,March,24379.7,1110.5,3700.0
0,2017,April,24387.4,1110.5,3710.7
51,2017,May,24409.0,1110.5,3689.1
...,...,...,...,...,...
62,2022,November,26325.0,0.0,3808.4
17,2022,December,26325.0,0.0,3808.4
31,2023,January,26330.2,0.0,3809.0
24,2023,February,26285.0,0.0,3808.6


In [114]:
#extract_nameplate_capacity()